# Tabla players

In [19]:
import pandas as pd
import numpy as np

Para crear esta dimensión utilizaremos las ids de la tabla de hechos de player_stats, las cuales necesitaran un tratado porque vendrán duplicadas, debido a que en la tabla de hechos las player_id se repiten debido a la presencia de los mismos jugadores en varias temporadas.

In [20]:
players= pd.read_csv('player_final_merged_with_stats.csv')

In [21]:
players.head()

,primary_key_composed,player_id,player_name,position,age,team_abbreviation,GP,GS,MP,FG,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,season_id,TOTAL TEAM
0,1_2021,1,Jaylen Adams,PG,24,[MIL],6,0,2.8,0.2,...,0.5,0.5,0.3,0.0,0.0,0.0,0.2,0.3,2021,[MIL]
1,2_2021,2,Bam Adebayo,C,23,[MIA],26,26,33.6,7.4,...,7.3,9.2,5.3,1.0,1.0,3.0,2.6,19.9,2021,[MIA/MIA/MIA]
2,3_2021,3,Ty-Shon Alexander,SG,22,[PHO],3,0,2.7,0.0,...,0.3,0.3,0.3,0.0,0.0,0.0,0.3,0.0,2021,[PHO]
3,4_2021,4,Jarrett Allen,C,22,[TOT],28,10,26.2,4.4,...,6.1,9.0,1.6,0.5,1.6,1.5,1.6,12.3,2021,[BRK/CLE/CLE/CLE]
4,5_2021,5,Al-Farouq Aminu,PF,30,[ORL],2,0,8.0,0.5,...,1.5,2.0,1.0,0.0,0.0,1.0,0.0,1.5,2021,[ORL]


In [22]:
# Seleccionamos solo las columnas 2 y 3 (considerando que los índices empiezan en 0)
df_seleccionado = players.iloc[:, [1, 2]]

# Mostrar el DataFrame resultante
print(df_seleccionado)

     player_id        player_name
0            1       Jaylen Adams
1            2        Bam Adebayo
2            3  Ty-Shon Alexander
3            4      Jarrett Allen
4            5    Al-Farouq Aminu
..         ...                ...
865        546      Justin Minaya
866        547       Kobi Simmons
867        471          Gabe York
868        548       RaiQuan Gray
869        549      Jacob Gilyard

[870 rows x 2 columns]


Verificamos los nulos y duplicados:

In [23]:
# PORCENTAJE DE VALORES NULOS
nulos_total = df_seleccionado.isnull().sum()
porcentaje_nulos = (nulos_total / len(df_seleccionado)) * 100

resumen_nulos = pd.DataFrame({
    'Valores Nulos': nulos_total,
    'Porcentaje Nulos': porcentaje_nulos
})

print(resumen_nulos)

             Valores Nulos  Porcentaje Nulos
player_id                0               0.0
player_name              0               0.0


In [24]:
duplicados = df_seleccionado.duplicated(keep=False)

# Filtra las filas duplicadas
datos_duplicados = df_seleccionado[duplicados]
print(datos_duplicados)

     player_id             player_name
1            2             Bam Adebayo
5            6   Giannis Antetokounmpo
7            8  Thanasis Antetokounmpo
8            9            Cole Anthony
9           10              OG Anunoby
..         ...                     ...
857        376             Xavier Moon
860         33        Jarrell Brantley
861        464       Lindell Wigginton
862        415          Jordan Schakel
867        471               Gabe York

[523 rows x 2 columns]


In [25]:
# Eliminamos las filas duplicadas basadas en la columna 'player_id'
df_unique = df_seleccionado.drop_duplicates(subset='player_id', keep='first')

# Verificamos si las filas duplicadas fueron eliminadas
print(f"Cantidad de filas después de eliminar duplicados: {df_unique.shape[0]}")

Cantidad de filas después de eliminar duplicados: 549


Ahora se verificara que las ID esten presentes correctamente en la tabla de hechos y de dimensión, para asegurar que no falte ninguna.

In [26]:
# Obtener los player_id únicos de cada DataFrame
player_ids_df1 = set(players['player_id'])
player_ids_df2 = set(df_unique['player_id'])

# Encontrar player_ids que están en df1 pero no en df2
ids_solo_en_df1 = player_ids_df1 - player_ids_df2

# Encontrar player_ids que están en df2 pero no en df1
ids_solo_en_df2 = player_ids_df2 - player_ids_df1

# Verificar si hay diferencias
if not ids_solo_en_df1 and not ids_solo_en_df2:
    print("Todos los player_id están presentes en ambos DataFrames.")
else:
    if ids_solo_en_df1:
        print(f"player_id en df1 pero no en df2: {ids_solo_en_df1}")
    if ids_solo_en_df2:
        print(f"player_id en df2 pero no en df1: {ids_solo_en_df2}")

Todos los player_id están presentes en ambos DataFrames.


A este nuevo dataframe le tenemos que agregar la columna de Salarios, que la traemos desde un .csv externo de kaggle.

In [27]:
salario = pd.read_csv('Salarios.csv')

In [28]:
salario.head()

,Unnamed: 0,Player,Salary,Temp
0,0,Stephen Curry,48070014,2023
1,1,John Wall,47345760,2023
2,2,Russell Westbrook,47080179,2023
3,3,LeBron James,44474988,2023
4,4,Kevin Durant,44119845,2023


In [29]:
# Dropeamos las columnas en los índices 0 y 3
df_salario = salario.drop(salario.columns[[0, 3]], axis=1)

# Mostrar el DataFrame resultante
print(df_salario)

                Player    Salary
0        Stephen Curry  48070014
1            John Wall  47345760
2    Russell Westbrook  47080179
3         LeBron James  44474988
4         Kevin Durant  44119845
..                 ...       ...
462      Justin Minaya     35096
463       Kobi Simmons     32795
464          Gabe York     32171
465       RaiQuan Gray      5849
466      Jacob Gilyard      5849

[467 rows x 2 columns]


In [30]:
df_unique

,player_id,player_name
0,1,Jaylen Adams
1,2,Bam Adebayo
2,3,Ty-Shon Alexander
3,4,Jarrett Allen
4,5,Al-Farouq Aminu
...,...,...
864,545,Jeenathan Williams
865,546,Justin Minaya
866,547,Kobi Simmons
868,548,RaiQuan Gray


In [31]:
# Renombrar la columna 'player_name' a 'Player' en df_unique para que coincida con df_salario
df_unique.rename(columns={'player_name': 'Player'}, inplace=True)

C:\Users\Agus\AppData\Local\Temp\ipykernel_1028\3808535591.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.rename(columns={'player_name': 'Player'}, inplace=True)


In [32]:
# Realiza el merge en base a la columna 'Player'
df_combined = pd.merge(df_unique, df_salario, on='Player', how='left')

In [33]:
# Rellenamos con 0 los datos de los jugadores que no sabemos su salario.
df_combined = df_combined.fillna(0)
df_combined['Salary'] = df_combined['Salary'].astype(int)

In [34]:
df_combined

,player_id,Player,Salary
0,1,Jaylen Adams,0
1,2,Bam Adebayo,30351780
2,3,Ty-Shon Alexander,0
3,4,Jarrett Allen,20000000
4,5,Al-Farouq Aminu,0
...,...,...,...
544,545,Jeenathan Williams,52644
545,546,Justin Minaya,35096
546,547,Kobi Simmons,32795
547,548,RaiQuan Gray,5849


In [35]:
df_combined.dtypes

player_id     int64
Player       object
Salary        int64
dtype: object

Guardamos la tabla de dimensión player_info finalizada.

In [36]:
df_combined.to_csv('player_info_final.csv', index=False)